In [1]:
import mrpt

In [2]:
import pandas as pd
import numpy as np
import torch as pt
import multiprocessing
#from bps import bps
from sembps.bps import bps
from torch.utils.data import Dataset, DataLoader
import os
from tqdm import tqdm
import h5py

In [3]:
MAIN_PATH = os.path.join('aptbps-code')
LOGS_PATH = os.path.join(MAIN_PATH, 'logs')
DATA_PATH = os.path.join(MAIN_PATH, 'data')

train_path = os.path.join(DATA_PATH, 'train')
hdf5_path = os.path.join(DATA_PATH, 'hdf5')
no_unlabeled_h5_path = os.path.join(DATA_PATH, 'no_unlabeled_hdf5')
hdf5_train_path = os.path.join(hdf5_path, 'train')
encoded_hdf5_path = os.path.join(DATA_PATH, 'tree_encoded_hdf5')


# All the clouds in the training dataset
train_files = [
    "bildstein_station1_xyz_intensity_rgb",
    "bildstein_station3_xyz_intensity_rgb",
    "bildstein_station5_xyz_intensity_rgb",
    "domfountain_station1_xyz_intensity_rgb",
    "domfountain_station2_xyz_intensity_rgb",
    "domfountain_station3_xyz_intensity_rgb",
    "neugasse_station1_xyz_intensity_rgb",
    "sg27_station1_intensity_rgb",
    "sg27_station2_intensity_rgb",
    "sg27_station4_intensity_rgb",
    "sg27_station5_intensity_rgb",
    "sg27_station9_intensity_rgb",
    "sg28_station4_intensity_rgb",
    "untermaederbrunnen_station1_xyz_intensity_rgb",
    "untermaederbrunnen_station3_xyz_intensity_rgb",
]

In [123]:
n_orig_points = 2048
n_bps_points = 512
n_dims = 3
radius = 1.5
random_seed = 13

In [124]:
# batch of 100 point clouds to convert
x = np.random.normal(size=[100, n_orig_points, 3])

# optional point cloud normalization to fit a unit sphere
x = bps.normalize(x)

In [125]:
from timeit import default_timer as timer
import time
from sklearn.neighbors import NearestNeighbors

In [126]:
n_clouds, n_points, n_dims = x.shape

basis_set = bps.generate_random_basis(n_bps_points, n_dims=n_dims, radius=radius, random_seed=random_seed)

n_bps_points = basis_set.shape[0]

x_bps = np.zeros([n_clouds, n_bps_points])
        
fid_lst = range(0, x.shape[0])

idx_bps = np.zeros([n_clouds, n_bps_points])

# demo (various cases with different tunings)
target_recall is the main tuning parameter; there's not much speed increase by lowering it though, might as well keep it high

In [94]:
target_recall = 0.9
n_neighbors = 1

# Single subcloud test

In [136]:
start = timer()

index = mrpt.MRPTIndex(x[0].astype(np.float32))

#index.build_autotune_sample(target_recall, n_neighbors)
    
#print(index.ann(basis_set.astype(np.float32), return_distances=True))
    
    # Find nn for each point in basis point cloud
for b_idx, b_point in enumerate(basis_set):
    b_point = b_point.astype(np.float32)
    nn_idx, nn_dist = index.exact_search(b_point, 1, return_distances=True)
    idx_bps[0][b_idx] = nn_idx[0]
    x_bps[0][b_idx] = nn_dist[0]

end = timer()
print(end-start)

0.009546916000545025


In [137]:
start = timer()

index = mrpt.MRPTIndex(x[0].astype(np.float32))

index.build_autotune_sample(target_recall, n_neighbors)
    
#print(index.ann(basis_set.astype(np.float32), return_distances=True))
    
    # Find nn for each point in basis point cloud
for b_idx, b_point in enumerate(basis_set):
    b_point = b_point.astype(np.float32)
    nn_idx, nn_dist = index.ann(b_point, return_distances=True)
    idx_bps[0][b_idx] = nn_idx[0]
    x_bps[0][b_idx] = nn_dist[0]

end = timer()
print(end-start)

0.021634887205436826


# Multiple clouds, iterate over points in basis point set

In [95]:
start = timer()

for fid in fid_lst:
    index = mrpt.MRPTIndex(x[fid].astype(np.float32))
    
    index.build_autotune_sample(target_recall, n_neighbors)
    
    # Find nn for each point in basis point cloud
    for b_idx, b_point in enumerate(basis_set):
        b_point = b_point.astype(np.float32)
        nn_idx, nn_dist = index.ann(b_point, return_distances=True)
        idx_bps[fid][b_idx] = nn_idx[0]
        x_bps[fid][b_idx] = nn_dist[0]

    
end = timer()
print(end-start)

1.6660681830253452


In [101]:
print(x_bps)

[[0.30067015 0.73955375 0.40114665 ... 0.05340877 0.48015454 0.38829109]
 [0.38480461 0.82479316 0.55298543 ... 0.03123789 0.53781599 0.33851123]
 [0.36253688 0.80617148 0.66118342 ... 0.03550712 0.46481341 0.36253688]
 ...
 [0.45241389 0.86386204 0.65280497 ... 0.05871286 0.57802105 0.36523324]
 [0.35403153 0.73373264 0.51478392 ... 0.02440229 0.43086687 0.3576676 ]
 [0.26779544 0.75354362 0.5570699  ... 0.02345088 0.47231674 0.26779544]]


# Multiple clouds, compare basis point set directly (no inner loop)

In [106]:
start = timer()

for fid in fid_lst:
    index = mrpt.MRPTIndex(x[fid].astype(np.float32))
    
    index.build_autotune_sample(target_recall, n_neighbors)
    
    nn_idx, nn_dist = index.ann(basis_set.astype(np.float32), return_distances=True)
    idx_bps[fid] = nn_idx[0]
    x_bps[fid] = nn_dist[0]
    
end = timer()
print(end-start)

1.3432215070351958


In [108]:
print(x_bps)
x_bps.shape

[[0.30067015 0.30067015 0.30067015 ... 0.30067015 0.30067015 0.30067015]
 [0.33851123 0.33851123 0.33851123 ... 0.33851123 0.33851123 0.33851123]
 [0.36253688 0.36253688 0.36253688 ... 0.36253688 0.36253688 0.36253688]
 ...
 [0.45298287 0.45298287 0.45298287 ... 0.45298287 0.45298287 0.45298287]
 [0.35403153 0.35403153 0.35403153 ... 0.35403153 0.35403153 0.35403153]
 [0.26779544 0.26779544 0.26779544 ... 0.26779544 0.26779544 0.26779544]]


(100, 512)

# As you can see above, there is a speed increase in comparing the basis point set directly, and the results are the same.
# Below: multiple clouds, no build_autotune_sample, exact_search instead of ann
as you can see it's way less efficient, so it's best to autotune.

In [154]:
start = timer()

for fid in fid_lst:
    index = mrpt.MRPTIndex(x[fid].astype(np.float32))
    
    #index.build_autotune_sample(target_recall, n_neighbors)
    
    nn_idx, nn_dist = index.exact_search(basis_set.astype(np.float32), n_bps_points, return_distances=True)
    idx_bps[fid] = nn_idx[0]
    x_bps[fid] = nn_dist[0]
    
end = timer()
print(end-start)

4.03850423800759


In [155]:
print(x_bps)
print(idx_bps)

[[0.26745006 0.36919162 0.37630063 ... 0.87939894 0.87967294 0.8799209 ]
 [0.14254008 0.22192255 0.27049926 ... 0.87376797 0.87380505 0.87392092]
 [0.25283483 0.31226045 0.38223329 ... 0.87014574 0.87017608 0.87175322]
 ...
 [0.39784214 0.39991084 0.4126842  ... 0.88317472 0.88376039 0.88379562]
 [0.22654429 0.28115085 0.28808853 ... 0.87038493 0.87072766 0.87121207]
 [0.29608193 0.31139612 0.38247636 ... 0.88602364 0.88663399 0.88773572]]
[[ 325. 1964. 1803. ... 1617. 1765. 1343.]
 [ 895. 1952. 1038. ... 1002.  536.  995.]
 [1943.  454.  169. ... 1395. 1954.  155.]
 ...
 [ 836.  575.  208. ...  324.  842. 1306.]
 [1812.  476. 1761. ...  911.  697. 1231.]
 [ 651.  104.  238. ... 1283.  143.  299.]]


In [145]:
start = timer()
for fid in fid_lst:
            nbrs = NearestNeighbors(n_neighbors=1, leaf_size=16, algorithm='kd_tree').fit(x[fid])
            fid_dist, npts_ix = nbrs.kneighbors(basis_set)
            x_bps[fid] = fid_dist.squeeze()
            idx_bps[fid] = npts_ix.squeeze()
end = timer()
print(end-start)

0.21964480890892446


In [146]:
x_bps

array([[0.26745007, 0.73740243, 0.49597505, ..., 0.04366947, 0.45987128,
        0.40547443],
       [0.14254003, 0.59741447, 0.55925921, ..., 0.04056936, 0.32275239,
        0.13695616],
       [0.25283479, 0.6802153 , 0.47804271, ..., 0.03196637, 0.42594227,
        0.33400218],
       ...,
       [0.39784214, 0.75019342, 0.60759885, ..., 0.03182879, 0.45885037,
        0.37687963],
       [0.22654428, 0.71636211, 0.56481734, ..., 0.03430374, 0.37982411,
        0.36013448],
       [0.29608191, 0.66290331, 0.42415639, ..., 0.03322053, 0.38666311,
        0.26885576]])